In [3]:
# Note, run this only once, all the following based on directory structure /content/gdrive/
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
# Run this only once into your DNABERT directory
%cd gdrive/MyDrive/NASA/DNABERT

!ls

/content/gdrive/MyDrive/NASA/DNABERT/examples
compute_result.py      run_finetune.py	sample_data
data_process_template  run_pretrain.py	scripts
requirements.txt       runs		visualize.py


In [7]:
!pip install transformers
!pip install evaluate

import pandas as pd
import torch
from transformers import AutoTokenizer, BertModel
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import evaluate
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00


In [ ]:
dna_data = pd.read_csv('data/dna_data.csv')
print(dna_data.shape)
dna_data.head()

(36000, 5)


,notation,DNA,mass,filename,label
0,A00654:48:HN52TDRXY:1:2101:5990:1000 2:N:0:AGT...,ATATTTATGGCTGGACTTGAACTTACTAAGTAGACCATGCTGGCCT...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,GLDS-466_metagenomics_RR10_FCS_VIV_WT_V1_R2_HR...,Wild Type
1,A00654:48:HN52TDRXY:1:2101:6840:1000 2:N:0:AGT...,CCTACGCTCAGCGAGGCGACTTTGAGAGATGCGCCGAAGAATCTTT...,"FFFF:F,F:FFFFFFFFFFF,FFF::FFFFFFFFF,,F,,FFF,F:...",GLDS-466_metagenomics_RR10_FCS_VIV_WT_V1_R2_HR...,Wild Type
2,A00654:48:HN52TDRXY:1:2101:9498:1000 2:N:0:AGT...,GCCTTGACCCATGCCTGATAAGGGAGGGCCCGGTCGACGCCCAGGA...,":FFFFFFFFFFFFFFFF:FFFFFFFFFF,FF:FFFFF:FFFFFFFF...",GLDS-466_metagenomics_RR10_FCS_VIV_WT_V1_R2_HR...,Wild Type
3,A00654:48:HN52TDRXY:1:2101:15067:1000 2:N:0:AG...,GGACAGGGCCGCAGCATATTCTCATTAAACGGCTGGCCGTCATGGT...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,GLDS-466_metagenomics_RR10_FCS_VIV_WT_V1_R2_HR...,Wild Type
4,A00654:48:HN52TDRXY:1:2101:1298:1016 2:N:0:AGT...,TCCTCCATCCATGCTTTGAATGTCTGGAACCCTGCCTCTGTGTTAC...,FFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFF:F...,GLDS-466_metagenomics_RR10_FCS_VIV_WT_V1_R2_HR...,Wild Type


In [ ]:
!git clone https://github.com/jerryji1993/DNABERT
%cd DNABERT
!python3 -m pip install --editable .
%cd examples
!python3 -m pip install -r requirements.txt

Cloning into 'DNABERT'...
remote: Enumerating objects: 774, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 774 (delta 56), reused 50 (delta 50), pack-reused 700
Receiving objects: 100% (774/774), 11.68 MiB | 7.11 MiB/s, done.
Resolving deltas: 100% (412/412), done.
Updating files: 100% (252/252), done.
/content/gdrive/MyDrive/NASA/DNABERT/examples/DNABERT
Obtaining file:///content/gdrive/MyDrive/NASA/DNABERT/examples/DNABERT
  Preparing metadata (setup.py) ... done
  Using cached tokenizers-0.5.0.tar.gz (64 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached boto3-1.28.62-py3-none-any.whl (135 kB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached botocore-1.31.62-py3-none-any.whl (11.2 MB)
  Using cach

In [ ]:
%cd examples

%env KMER=6
%env TRAIN_FILE=./sample_data/pre/dna_str.txt # This is the path to your dna txt generated by data processing steps in README
%env TEST_FILE=./sample_data/pre/dna_str.txt # This is the path to your dna txt generated by data processing steps in README
%env SOURCE=/content/gdrive/MyDrive/NASA/DNABERT # Path to your source
%env OUTPUT_PATH=output$KMER


# Notice:
# MUST set your tokenizer name to vocab.txt o/w there will be issue will Huggingface validation

!python run_pretrain.py \
    --output_dir=$OUTPUT_PATH \
    --model_type=dna \
    --tokenizer_name=$SOURCE/src/transformers/dnabert-config/bert-config-$KMER/vocab.txt \ 
    --config_name=$SOURCE/src/transformers/dnabert-config/bert-config-$KMER/config.json \
    --do_train \
    --train_data_file=$TRAIN_FILE \
    --do_eval \
    --eval_data_file=$TEST_FILE \
    --mlm \
    --gradient_accumulation_steps 25 \
    --per_gpu_train_batch_size 10 \
    --per_gpu_eval_batch_size 6 \
    --save_steps 500 \  
    --save_total_limit 20 \
    --max_steps 200000 \
    --evaluate_during_training \
    --logging_steps 500 \
    --line_by_line \
    --learning_rate 4e-4 \
    --block_size 64 \
    --adam_epsilon 1e-6 \
    --weight_decay 0.01 \
    --beta1 0.9 \
    --beta2 0.98 \
    --mlm_probability 0.025 \
    --warmup_steps 10000 \
    --overwrite_output_dir \
    --n_process 24

[Errno 2] No such file or directory: 'examples'
/content/gdrive/MyDrive/NASA/DNABERT/examples
env: KMER=6
env: TRAIN_FILE=./sample_data/pre/dna_str.txt
env: TEST_FILE=./sample_data/pre/dna_str.txt
env: SOURCE=/content/gdrive/MyDrive/NASA/DNABERT
env: OUTPUT_PATH=output$KMER
2023-10-08 14:25:35.687153: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
10/08/2023 14:25:36 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1946: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
10/08/2023 14:25:36 - INFO - __main__ -   Training new model from scratch
10/08/2023 14:25:38 - INFO - __main__ -   Training/evaluation parameters Namespace

In [ ]:
%ls ./sample_data

ft/  pre/


  Using cached transformers-4.34.0-py3-none-any.whl (7.7 MB)
